In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('./sample_data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    prob = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, p=prob, replace=False)
    
    return recs.tolist()

In [5]:
result = data_train.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']  
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head(2)

,user_id,actual
0,1,"[825123, 831447, 840361, 845307, 852014, 85498..."
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55..."


In [6]:

def get_weight(train):
    item_weight = train.groupby('item_id')['sales_value'].mean().reset_index()
    item_weight.sales_value *= 100000
    item_weight.sales_value += 1
    item_weight['weight'] = np.log(item_weight['sales_value'])
    item_weight.loc[np.isinf(item_weight['weight'])] = 0
    item_weight.weight /= item_weight.weight.sum()
    del item_weight['sales_value']

    return item_weight

In [7]:
%%time
result['random_recomendation'] = result['user_id'].apply(lambda x:  weighted_random_recommendation(get_weight(data_train)))

CPU times: user 4min 56s, sys: 4.65 s, total: 5min
Wall time: 4min 59s


In [8]:
result.head(2)

,user_id,actual,random_recomendation
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[860176, 10311475, 5583222, 1010949, 9487900]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[6533350, 9270345, 6395907, 2483014, 1106753]"


### Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [9]:
import numpy as np
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [10]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [11]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values
matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))
userid_to_id = dict(zip(userids, matrix_userids))
itemid_to_id = dict(zip(itemids, matrix_itemids))
id_to_itemid = dict(zip(matrix_itemids, itemids))
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [12]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [13]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера (почему рекомендация не зависит от юзера?)
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 128 ms, sys: 78.1 ms, total: 206 ms
Wall time: 203 ms


In [14]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/86865 [00:00<?, ?it/s]

CPU times: user 38.7 s, sys: 282 ms, total: 39 s
Wall time: 31.7 s


In [15]:
result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

In [16]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/86865 [00:00<?, ?it/s]

CPU times: user 33.3 s, sys: 218 ms, total: 33.5 s
Wall time: 26 s


In [17]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 294 ms, sys: 983 µs, total: 295 ms
Wall time: 295 ms


In [18]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

  0%|          | 0/86865 [00:00<?, ?it/s]

CPU times: user 26.3 s, sys: 195 ms, total: 26.4 s
Wall time: 18.9 s


In [19]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 99.2 ms, sys: 973 µs, total: 100 ms
Wall time: 101 ms


In [20]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/86865 [00:00<?, ?it/s]

CPU times: user 33.4 s, sys: 236 ms, total: 33.6 s
Wall time: 26.1 s


In [21]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 173 ms, sys: 29 µs, total: 173 ms
Wall time: 175 ms


In [22]:
rows_list = []
rows_list.append(['random_recomendation', result.apply(lambda x: precision_at_k(x['random_recomendation'], x['actual'],  5), axis=1).mean()])
rows_list.append(['itemitem', result.apply(lambda x: precision_at_k(x['itemitem'], x['actual'],  5), axis=1).mean()])
rows_list.append(['tfidf', result.apply(lambda x: precision_at_k(x['tfidf'], x['actual'],  5), axis=1).mean()])
rows_list.append(['popular_recommendation', result.apply(lambda x: precision_at_k(x['popular_recommendation'], x['actual'],  5), axis=1).mean()])
rows_list.append(['cosine', result.apply(lambda x: precision_at_k(x['cosine'], x['actual'],  5), axis=1).mean()])
rows_list.append(['own_purchases', result.apply(lambda x: precision_at_k(x['own_purchases'], x['actual'],  5), axis=1).mean()])
metrics = pd.DataFrame(rows_list, columns = ['Algorithm', 'Value'])
metrics

,Algorithm,Value
0,random_recomendation,0.006242
1,itemitem,0.657463
2,tfidf,0.864426
3,popular_recommendation,0.393037
4,cosine,0.796078
5,own_purchases,1.000000
